# **Infer `Cellmask` and `Nucleus`**

***Prior to this notebook, you should have already run through [1.0_image_setup](1.0_image_setup.ipynb).***

### ➡️ **Input:**
In this workflow, a single or multi-channel confocal microscopy image of fluorescently tagged organelles will be used to "infer" (or segment) the cell and nucleus masks. The following setup is recommended for this pipeline:

| **Imaging Requirements**|[**Masks Workflow (C)**](./1.1c_infer_masks_from-composite_neuron_with_pm.ipynb)|
| :------------------------------------- |  :------:  |
| Nuclei Marker                          |     ✔     |
| Cell Membrane Marker                   |     ✔     |
| Cytoplasmic Organelles                 |      ✔     |
| Number of cells per image              |  Single or Multiple |
| Applicable with sample data         | Neuron_2|

*If your images are not compatible with this setup, consider the other 1.1_Masks Workflows available:*

| **Imaging Requirements**|[**Masks Workflow**](./1.1_infer_masks_from-composite_with_nuc.ipynb)|[**Masks Workflow (A)**](./1.1a_infer_masks_from-composite_single_cell.ipynb)|[**Masks Workflow (B)**](./1.1b_infer_masks_from-composite_multiple-cells.ipynb)|
| :------------------------------------- |  :------:  |  :------:  |  :------:  |
| Nuclei Marker                          |     ✔     |      ✘     |     ✘     |
| Cell Membrane Marker                   |     ✘     |      ✘     |     ✘     |
| Cytoplasmic Organelles                 |     ✔     |      ✔     |     ✔     |
| Number of cells per image              |  Single or Multiple  |   Single   |  Multiple |
| Applicable with sample data         |  ✘  |   Neuron_1   |  Astrocyte |

### **Included in this Notebook:**
1. **Infer *nucleus*** - Segment the ***nuclei*** from a single channel (nuclei), then select a single **nucleus** via a search image. This will be necessary to determine the other subcellular compartments. The **nucleus** will also be used to seed the instance segmentation of the ***cell*** area (***cellmask***).

2. **Infer *cellmask*** - Segment the cell area (the ***cellmask***) from a composite image of multiple organelle markers and the ***nucleus*** object. The **cellmask** will be necessary for determining which organelles are in which cell.
-----

### 👣 **Summary of steps**

### !!!ADD DIAGRAM HERE!!!

➡️ **EXTRACTION**
- **`STEP 1`** - Create composite image - I

    - determine weight to apply to each channel of the intensity image (w# = user input)
    - choose whether to invert plasma membrane or not (invert_PM = user input)
    - select plasma membrane channel (pm_channel = user input)

- **`STEP 2`** - Create composite image - II

    - determine weight to apply to each channel of the intensity image (w# = user input)
    - choose whether to invert plasma membrane or not (invert_PM = user input)
    - select plasma membrane channel (pm_Channel = user input)

**PRE-PROCESSING**
- **`STEP 3`** - Close gaps and apply filters – I

    - apply closing algorithm or scharr edge detection (method = user input)
    - set the footprint size for the closing algorithm (size = user input)
    
- **`STEP 4`** - Close gaps and apply filters – II

    - apply closing algorithm or scharr edge detection (method = user input)
    - set the footprint size for the closing algorithm (size = user input)
 
**CORE PROCESSING**
- **`STEP 5`** - Threshold object and restrict to PM - I

    - apply MO thresholding method from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (threshold options = user input)
    - bind resulting segmentation to inverted plasma membrane

- **`STEP 6`** - Threshold object and restrict to PM - II

    - apply MO thresholding method from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (threshold options = user input)
    - bind resulting segmentation to inverted plasma membrane

**POST-PROCESSING**
- **`STEP 7`** - Find the nucleus corresponding to the cell

    - (A) select single channel containing the nuclei marker (channel number = user input)
    - (B) rescale intensity of composite image (min=0, max=1); apply median (median size = user input) and gaussian filter (sigma = user input)
    - (C) log transform image and calculate Li's minimum cross entropy threshold value; apply threshold to image (thresholding options = user input)
    - (D) fill holes (hole size = user input) and remove small objects (object size = user input)
    - (E) label nuclei objects
    - (F) use search image to select single nucleus

- **`STEP 8`** - Combine nucleus and thresh and fill holes - I

    - select the type of footprint to be used in nucleus dilation (method = user input)
    - set the footprint size for nucleus dilation (size = user input)
    - fill holes (hole size = user input)
    - remove small objects (object size = user input)
    - combine nucleus and mask_I

- **`STEP 9`** - Combine nucleus and thresh and fill holes - II

    - select the type of footprint to be used in nucleus dilation (method = user input)
    - set the footprint size for nucleus dilation (size = user input)
    - fill holes (hole size = user input)
    - remove small objects (object size = user input)
    - combine nucleus and mask_II

**POST-POST-PROCESSING**
- **`STEP 10`** - Watershed for cellmask

    - perform water-shedding algorithm on the mask_I and mask_II segmentations using the nucleus as the seed (watershed_method = user input)
    - combine resulting segmentations for rough cellmask
    - apply binary closing to rough cellmask, dilation -> erosion (method = user input)
    - set the footprint size for nucleus dilation (size = user input)
    - fill holes (hole size = user input)

**OUTPUT** ➡️ 
- **`STEP 11`** - Stack masks

    - Stack masks in order of nucleus, then cellmask


---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`1.0_image_setup`](1.0_image_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

#### &#x1F3C3; **Run code; no user input required**

In [1]:
from pathlib import Path
import os

import numpy as np
import pandas as pd
import napari
from napari.utils.notebook_display import nbscreenshot

from skimage.morphology import binary_dilation, binary_erosion

from infer_subc.core.file_io import (read_czi_image, 
                                     export_inferred_organelle,
                                     list_image_files,
                                     sample_input)
from infer_subc.core.img import *
from infer_subc.organelles.membrane import *

%load_ext autoreload
%autoreload 2

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `im_type`, `data_root_path`, `in_data_path`, and `out_data_path`.

In [2]:
### USER INPUT REQUIRED ###
# If using the sample data, set cell_type to "neuron_2":
# If not using the sample data, set cell_type to None
sample_data_type = "neuron_2"

# If you are not using the sample data, please edit "USER SPECIFIED" as necessary.
## Define the path to the directory that contains the input image folder.
data_root_path = Path("USER SPECIFIED")

# Specify the file type of your raw data that will be analyzed. Ex) ".czi" or ".tiff"
im_type = "USER SPECIFIED"

## Specify which subfolder that contains the input data
in_data_path = data_root_path / "USER SPECIFIED"

## Specify the output folder to save the segmentation outputs if.
## If its not already created, the code below will creat it for you
out_data_path = data_root_path / "USER SPECIFIED"

#### &#x1F3C3; **Run code; no user input required**

In [3]:
# If sample_data_type is set "neuron_2", then the sample data is used and the directories are set
if sample_data_type != None:
    data_root_path, im_type, in_data_path, out_data_path = sample_input(sample_data_type)

# list files in the input folder
img_file_list = list_image_files(in_data_path,im_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":img_file_list})

,Image Name
0,c:\Users\redre\Documents\CohenLab\scohen_lab_repo\infer-subc\sample_data\example_neuron_2\raw\20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome.tiff


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [4]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [5]:
# load image and metadata
test_img_name = img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# metadata
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']
print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

# open viewer and add images
viewer = napari.Viewer()
for i in list(range(len(channel_names))):
    viewer.add_image(img_data[i],
                     scale=scale,
                     name=f"Channel {i}")
viewer.grid.enabled = True
viewer.reset_view()
print("\nProceed to Napari window to view your selected image.")

Metadata information
File path: c:\Users\redre\Documents\CohenLab\scohen_lab_repo\infer-subc\sample_data\example_neuron_2\raw\20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome.tiff
Channel 0 name: 20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome :: Channel:0
Channel 1 name: 20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome :: Channel:1
Channel 2 name: 20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome :: Channel:2
Channel 3 name: 20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome :: Channel:3
Channel 4 name: 20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome :: Channel:4
Channel 5 name: 20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome :: Channel:5
Channel 6 name: 20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome :: Channel:6
Channel 7 name: 20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome :: Channel:7
Scale (ZYX): (0.389118, 0.07064, 0.07064)
Channel axis: 0

Proceed to Napari window to view your selected image.


---------------------
## **masks_C**

The **masks_C** workflow operates by running two parallel sequences and combines their outputs to produce a **nucleus** and **cellmask** segmentation. **`Steps 7`** and **`10`** produce the **nucleus** and **cellmask** segmentations respectfully, using output from both sequences. However, **`step 7`** specifically uses output from either **`step 5`** or **`step 6`** (not both), this will be explained in greater detail later in the notebook. To conclude the workflow, **`step 11`** exports the **nucleus** and **cellmask** as one multichannel stack. 

The morphological aspects of the cell that each sequence captures are at the discretion of the user. Below is a basic diagram which visualizes the structure of the masks_C workflow.

```mermaid

masks_C workflow

         Sequence I                   Export
         
         1 --→ 3 --→ 5 --→ 8            7
       ↗             ⇘  ↗   ↘           ↘
    0 <                7      10           > 11
       ↘             ⇗  ↘   ↗           ↗
         2 --→ 4 --→ 6 --→ 9           10
         
         Sequence II


0 - raw image
1 - composite image I
2 - composite image II
3 - composite mask_I
4 - composite mask_II
5 - mask_I segmentation
6 - mask_II segmentation
7 - nucleus segmentation
8 - cellmask_I segmentation
9 - cellmask_II segmentation
10 - cellmask segmentation
11 - masks_C

>
>**If using the Sample Data 📂**:
>
>The steps ending in **I** refer to the sequence of steps that focus on capturing the cell's outer edges. By contrast, the steps ending in **II** refer to the sequence of steps that focus on capturing the interior. By detecting different regions of the cell, the quality of the comprehensive cellmask segmentation is improved.

-----

## **EXTRACTION**

### **`STEP 1` - Create composite image - I**

&#x1F453; **FYI:** This code block creates the **first** composite image of the organelle channels. The intensity of each channel is combined with specified weights.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `Invert_PM`: "True" inverts the intensities of the plasma membrane channel when added to the composite image; "False" leaves the intensity values as is when added to the composite image
- `PM_channel`: the index of the channel containing your plasma membrane label. Image indexing begins with 0, not 1. Reference the channel numbers indicated in the Napari window for easy reference.
- `weight_ch0`: the weight of channel 0 (the first channel); a value of 0 will exclude this channel from the compostite; large values will cause this channel to be more prominent in the final composite image
- `weight_ch1`: the weight of channel 1
- `weight_ch2`: the weight of channel 2
- `weight_ch3`: the weight of channel 3
- `weight_ch4`: the weight of channel 4
- `weight_ch5`: the weight of channel 5
- `weight_ch6`: the weight of channel 6
- `weight_ch7`: the weight of channel 7
- `weight_ch8`: the weight of channel 8
- `weight_ch9`: the weight of channel 9

In [6]:
#### USER INPUT REQUIRED ###
invert_pm_I = True
PM_CH = 5
w0_I = 20
w1_I = 10
w2_I = 5
w3_I = 15
w4_I = 12
w5_I = 15
w6_I = 0
w7_I = 0
w8_I = 0
w9_I = 0

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block creates the composite based on the settings above. The image is then added to Napari as a new layer for visual comparison to the input image. 

Use the Napari viewer to iteratively adjust the weights selected above.

In [7]:
# make aggregate and optionally rescale
struct_img_raw_I = membrane_composite(img_data,
                                weight_ch0= w0_I,
                                weight_ch1= w1_I,
                                weight_ch2= w2_I,
                                weight_ch3= w3_I,
                                weight_ch4= w4_I,
                                weight_ch5= w5_I,
                                weight_ch6= w6_I,
                                weight_ch7= w7_I,
                                weight_ch8= w8_I,
                                weight_ch9= w9_I,
                                Invert_PM = invert_pm_I,
                                PM_Channel = PM_CH)

# adding image to Napari as a new layer
viewer.layers.clear()
viewer.grid.enabled = False
viewer.reset_view()
viewer.add_image(img_data, scale=scale)
viewer.add_image(struct_img_raw_I, scale=scale, name="1-Cell: Create composite I")

<Image layer '1-Cell: Create composite I' at 0x1d6e31c9150>

### **`STEP 2` - Create composite image - II**

&#x1F453; **FYI:** This code block creates the **second** composite image of the organelle channels. The intensity of each channel is combined with specified weights.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `Invert_PM`: "True" inverts the intensities of the plasma membrane channel when added to the composite image; "False" leaves the intensity values as is when added to the composite image
- `PM_channel`: the index of the channel containing your plasma membrane label. Image indexing begins with 0, not 1. Reference the channel numbers indicated in the Napari window for easy reference.
- `weight_ch0`: the weight of channel 0 (the first channel); a value of 0 will exclude this channel from the compostite; large values will cause this channel to be more prominent in the final composite image
- `weight_ch1`: the weight of channel 1
- `weight_ch2`: the weight of channel 2
- `weight_ch3`: the weight of channel 3
- `weight_ch4`: the weight of channel 4
- `weight_ch5`: the weight of channel 5
- `weight_ch6`: the weight of channel 6
- `weight_ch7`: the weight of channel 7
- `weight_ch8`: the weight of channel 8
- `weight_ch9`: the weight of channel 9

In [8]:
#### USER INPUT REQUIRED ###
invert_pm_II = False
PM_CH = 5
w0_II = 20
w1_II = 10
w2_II = 5
w3_II = 15
w4_II = 12
w5_II = 0
w6_II = 0
w7_II = 0
w8_II = 0
w9_II = 0

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block creates the composite based on the settings above. The image is then added to Napari as a new layer for visual comparison to the input image. 

Use the Napari viewer to iteratively adjust the weights selected above.

In [9]:
# make aggregate and optionally rescale
struct_img_raw_II = membrane_composite(img_data,
                                weight_ch0= w0_II,
                                weight_ch1= w1_II,
                                weight_ch2= w2_II,
                                weight_ch3= w3_II,
                                weight_ch4= w4_II,
                                weight_ch5= w5_II,
                                weight_ch6= w6_II,
                                weight_ch7= w7_II,
                                weight_ch8= w8_II,
                                weight_ch9= w9_II,
                                Invert_PM = invert_pm_II,
                                PM_Channel = PM_CH)

# adding image to Napari as a new layer
viewer.add_image(struct_img_raw_II, scale=scale, name="2-Cell: Create composite II")

<Image layer '2-Cell: Create composite II' at 0x1d6da69f0d0>

-----
## **PRE-PROCESSING**

### **`STEP 3` - Close gaps and apply filters – I**

&#x1F453; **FYI:** Based on the selected method, this step can either apply a closing algorithm (dilate -> erode) to remove small holes and cracks, or apply a log transform and [Scharr](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.scharr) edge dectection . If Method = `"Ball"` or `"Disk"`, then a closing algorithm will be performed on the composite I image using the footprint specified by the method and size. If Method = `"Scharr"`, then the log-transformation and [Scharr](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.scharr) edge detection is applied to the composite I image. This also means that the size parameter is disregarded.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `Method`: "Ball" or "Disk" applies the closing algorithm, "Scharr" applies Scharr edge detection
- `Size`: size of the footprint used in the closing algorithm

In [10]:
#### USER INPUT REQUIRED ###
method_I = "Disk"
size_I = 5

In [11]:
# apply closing or the scharr edge detection filter
composite_mask_I = close_and_filter(struct_img_raw_I,
                                    Method = method_I,
                                    Size = size_I)

# adding image to Napari as a new layer
viewer.add_image(composite_mask_I, scale=scale, name="3-Cell: Closing/Scharr I")

<Image layer '3-Cell: Closing/Scharr I' at 0x1d6e1b69c60>

### **`STEP 4` - Close gaps and apply filters – II**

&#x1F453; **FYI:** Based on the selected method, this step can either apply a closing algorithm (dilate -> erode) to remove small holes and cracks, or apply a log transform and [Scharr](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.scharr) edge dectection . If Method = `"Ball"` or `"Disk"`, then a closing algorithm will be performed on the composite II image using the footprint specified by the method. If Method = `"Scharr"`, then the log-transformation and [Scharr](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.scharr) edge detection is applied to the composite II image. This also means that the `size` parameter is disregarded.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `Method`: "Ball" or "Disk" applies the closing algorithm, "Scharr" applies Scharr edge detection
- `Size`: size of the footprint used in the closing algorithm

In [12]:
#### USER INPUT REQUIRED ###
method_II = "Scharr"
size_II = 0

In [14]:
# apply closing or the scharr edge detection filter
composite_mask_II = close_and_filter(struct_img_raw_II,
                                    Method = method_II,
                                    Size = size_II)

# adding image to Napari as a new layer
viewer.add_image(composite_mask_II, scale=scale, name="4-Cell: Closing/Scharr II")

<Image layer '4-Cell: Closing/Scharr II' at 0x1d6e2bcc5e0>

-----
## **CORE-PROCESSING**

### **`STEP 5` - Threshold object and restrict to PM - I**

&#x1F453; **FYI:** This code block creates a semantic segmentation of a portion of the cell area (capturing certain aspects of the cell). `Semantic segmentation` is the process of deciding whether a pixel/voxel should be included in an object (labeled with a value of 1) or should be considered as part of the background (labeled with a value of 0). When performed on the output of Step 3, this results in the mask_I segmentation. A semantic segmentation does not discern individual objects from one another.

The masked_object_filter utilizes the 'MO' filter from the [`aics-segmentation`](https://github.com/AllenCell/aics-segmentation) package. AICS documentation states: "The algorithm is a hybrid thresholding method combining two levels of thresholds. The steps are: [1] a global threshold is calculated, [2] extract each individual connected componet after applying the global threshold, [3] remove small objects, [4] within each remaining object, a local Otsu threshold is calculated and applied with an optional local threshold adjustment ratio (to make the segmentation more and less conservative). An extra check can be used in step [4], which requires the local Otsu threshold larger than 1/3 of global Otsu threhsold and otherwise this connected component is discarded."

If Bind_To_PM = `"True"`, the plasma membrane channel can be log-transformed and have an Otsu threshold can be applied to produce a plasma membrane segmentation. This can then be inverted and combined with the intial mask_I segmentation to create a mask_I segmentation that is restricted to the plasma membrane boundary. If Bind_To_PM = `"False"`, this additional step is skipped.

#### &#x1F6D1; &#x270D; **User Input Required:**

- `global_method`: the type of thresholding method to be performed globally (MO)
- `cutoff_size`: the minimum object size to advance to the local thresholding step (MO)
- `local_adjust`: the ratio applied to the local Otsu threshold (MO)
- `Bind_To_PM`: Whether to restrict the mask_I segmentation to the inverted plasma membrane segmentation
- `PM_Channel`: the index of the plasma membrane channel (Please note that the first channel is indexed as 0)
- `Thresh_Adj`: The ratio applied to the Otsu threshold of the plasma membrane


In [15]:
#### USER INPUT REQUIRED ###
global_method_I = "triangle"
cutoff_size_I = 413
local_adj_I = 0.533
bind_to_pm_I = False
PM_CH = 5
thresh_adj_I = 1.0

In [16]:
# Create mask I segmentation (optional to bind to plasma membrane)
mask_I_segmentation = masked_object_thresh_bind_pm(
    img_data,
    composite_mask_I,
    Global_Method = global_method_I,
    Cutoff_Size = cutoff_size_I,
    Local_Adjust = local_adj_I,
    Bind_to_PM = bind_to_pm_I,
    PM_Channel = PM_CH,
    Thresh_Adj = thresh_adj_I)

# adding image to Napari as a new layer
viewer.add_image(mask_I_segmentation, scale=scale, name="5-Cell: MO Thresholding I")

<Image layer '5-Cell: MO Thresholding I' at 0x1d6e5bb6920>

### **`STEP 6` - Threshold object and restrict to PM - II**

&#x1F453; **FYI:** TThis code block creates a semantic segmentation of a portion of the cell area (capturing certain aspects of the cell). `Semantic segmentation` is the process of deciding whether a pixel/voxel should be included in an object (labeled with a value of 1) or should be considered as part of the background (labeled with a value of 0). When performed on the output of Step 4, this results in the mask_II segmentation. A semantic segmentation does not discern individual objects from one another.

The masked_object_filter utilizes the 'MO' filter from the [`aics-segmentation`](https://github.com/AllenCell/aics-segmentation) package. AICS documentation states: "The algorithm is a hybrid thresholding method combining two levels of thresholds. The steps are: [1] a global threshold is calculated, [2] extract each individual connected componet after applying the global threshold, [3] remove small objects, [4] within each remaining object, a local Otsu threshold is calculated and applied with an optional local threshold adjustment ratio (to make the segmentation more and less conservative). An extra check can be used in step [4], which requires the local Otsu threshold larger than 1/3 of global Otsu threhsold and otherwise this connected component is discarded."

If Bind_To_PM = `"True"`, the plasma membrane channel can be log-transformed and have an Otsu threshold can be applied to produce a plasma membrane segmentation. This can then be inverted and combined with the intial mask_II segmentation to create a mask_II segmentation that is restricted to the plasma membrane boundary. If Bind_To_PM = `"False"`, this additional step is skipped.

#### &#x1F6D1; &#x270D; **User Input Required:**

- `global_method`: the type of thresholding method to be performed globally (MO)
- `cutoff_size`: the minimum object size to advance to the local thresholding step (MO)
- `local_adjust`: the ratio applied to the local Otsu threshold (MO)
- `Bind_To_PM`: Whether to restrict the mask_II segmentation to the inverted plasma membrane segmentation
- `PM_Channel`: the index of the plasma membrane channel (Please note that the first channel is indexed as 0)
- `Thresh_Adj`: The ratio applied to the Otsu threshold of the plasma membrane


In [17]:
#### USER INPUT REQUIRED ###
global_method_II = "ave_tri_med"
cutoff_size_II = 787
local_adj_II = 1.0
bind_to_pm_II = True
PM_CH = 5
thresh_adj_II = 10.0

In [18]:
# Create mask I segmentation (optional to bind to plasma membrane)
mask_II_segmentation = masked_object_thresh_bind_pm(
    img_data,
    composite_mask_II,
    Global_Method = global_method_II,
    Cutoff_Size = cutoff_size_II,
    Local_Adjust = local_adj_II,
    Bind_to_PM = bind_to_pm_II,
    PM_Channel = PM_CH,
    Thresh_Adj = thresh_adj_II)

# adding image to Napari as a new layer
viewer.add_image(mask_II_segmentation, scale=scale, name="6-Cell: MO Thresholding II")

<Image layer '6-Cell: MO Thresholding II' at 0x1d6820518a0>

-----
## **POST-PROCESSING**

### **`STEP 7` - Find the nucleus corresponding to the cell**

&#x1F453; **FYI:** This step creates a semantic segmentation of the nucleus using the nuclei channel. First the nuclei intensities are blurred and then log transformed, leading to a nuclei segmentation using [Li's Minimum Cross Entropy](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_li) method. If there are multiple nuclei objects, then a search image is used to determine which nucleus is the target. The Search_Img can be set to `"Img 5"` (mask_I segmentation) or `"Img 6"` (mask_II segmentation). The nucleus label with the most overlap with the search image is selected as the nucleus object.

#### **`A. Select channel`**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify which channel includes your nuclei label:
- `NUC_CH`: the index of the channel containing your nuclei label. Image indexing begins with 0, not 1. Reference the channel numbers indicated in the Napari window for easy reference.

In [19]:
#### USER INPUT REQUIRED ###
NUC_CH = 7

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block extracts the nuclei channel from your multi-channel image. It will be the only part of the image used in the rest of this workflow. The single nuclei channel is added to the Napari viewer.

In [20]:
# select channel
raw_nuclei = select_channel_from_raw(img_data, NUC_CH)

# add single channel as a new layer
viewer.add_image(raw_nuclei, scale=scale, name="7-Nuc: Extract Channel")

<Image layer '7-Nuc: Extract Channel' at 0x1d682153610>

#### **`B. Rescale and smooth image`**

&#x1F453; **FYI:** This code block rescales the image so that the pixel/voxel with the highest intensity is set to 1 and the one with the lowest intensity is set to 0. The image is then *optionally* smoothed using a Gaussian and/or median filter. 

<mark> Include more information on the Gaussian and median filtering methods here </mark>

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the amount of filter to use for each method. Higher values indicate more smoothing:
- `med_filter_size`: the size of the median filter to apply; if 0 is used, no filter will be applied
- `gaussian_smoothing_sigma`: the sigma to apply in the Gaussian filtering step; if 0 is used, no filter will be applied

In [21]:
#### USER INPUT REQUIRED ###
nuc_med_filter_size = 10
nuc_gaussian_smoothing_sigma = 3.0

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block rescales the image and applies the specified median and Gaussian filters. The image is then added to Napari as a new layer for visual comparison to the input image. 

Use the Napari viewer to iteratively adjust the smoothing settings selected above.

In [22]:
# rescaling and smoothing input image
nuclei =  scale_and_smooth(raw_nuclei,
                           median_size = nuc_med_filter_size, 
                           gauss_sigma = nuc_gaussian_smoothing_sigma)

# adding image to Napari as a new layer
viewer.add_image(nuclei, scale=scale, name="7-Nuc: Rescale and Smooth")

<Image layer '7-Nuc: Rescale and Smooth' at 0x1d6820db520>

#### **`C. Apply log transform and threshold`**

&#x1F453; **FYI:** This code block applies a log transform and creates semantic segmentation using [Li's Minimum Cross Entropy](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_li) method. `Semantic segmentation` is the process of deciding whether a pixel/voxel should be included in an object (labeled with a value of 1) or should be considered as part of the background (labeled with a value of 0). A semantic segmentation does not discern individual objects from one another.


#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information:
- `threshold_factor`: adjustment to make to the intial local threshold; larger values make the segmentation more stringent (less area included)
- `thresh_min`: minimum bound of the local threshold
- `thresh_max`: maximum bound of the local threshold

In [23]:
#### USER INPUT REQUIRED ###
nuc_threshold_factor = 0.9
nuc_thresh_min = .12
nuc_thresh_max = 1.0

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block executes the log transform and threshold using the settings above.

Use the Napari viewer to iteratively adjust the filter settings as needed.

In [24]:
# log transform the image, calculate the threshold value using Li minimum cross entropy method, inverse log transform the value
li_thresholded = apply_log_li_threshold(nuclei, 
                                        thresh_factor=nuc_threshold_factor, 
                                        thresh_min=nuc_thresh_min, 
                                        thresh_max=nuc_thresh_max)

# adding image to Napari as a new layer
viewer.add_image(li_thresholded, scale=scale, name="7-Nuc: Threshold", opacity=0.3, colormap="cyan", blending='additive')

<Image layer '7-Nuc: Threshold' at 0x1d682153df0>

#### **`D. Remove small holes and objects`**

&#x1F453; **FYI:** This code block cleans up the semantic segmentation by filling small holes and/or removing small objects that can be considered errors in the initial segmentation. 

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `hole_min_width`: the width of the smallest hole to be filled
- `hole_max_width`: the width of the largest hole to be filled
- `small_object_width`: the width of the largest object to be removed; any object smaller than this size will be removed
- `fill_filter_method`: "3D" processes the image taking into account segmentation values in three dimensions (XYZ); "slice-by-slice" processes each Z-slice in the image separately, not considering the segmentation results in higher or lower Z planes.

In [25]:
#### USER INPUT REQUIRED ###
nuc_hole_min_width = 0
nuc_hole_max_width = 25  

nuc_small_object_width = 15

nuc_fill_filter_method = "3D"

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block executes the dot filter using the settings above.

Use the Napari viewer to iteratively adjust the filter settings as needed. 

*Hint: white pixels/voxels are the ones remaining after this step*

In [26]:
# combine the above functions into one for downstream use in plugin
cleaned_img = fill_and_filter_linear_size(li_thresholded, 
                                           hole_min=nuc_hole_min_width, 
                                           hole_max=nuc_hole_max_width, 
                                           min_size=nuc_small_object_width,
                                           method=nuc_fill_filter_method)

# adding image to Napari as a new layer
viewer.add_image(cleaned_img, scale=scale, name="7-Nuc: Fill holes and remove small objects", colormap="magenta", blending="additive")

<Image layer '7-Nuc: Fill holes and remove small objects' at 0x1d6e4a45c60>

#### **`E. - Label objects`**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block takes the semantic segmentation and creates an `instance segmentation`. In this output, each individual object in the image is given a unique ID number. The background pixels/voxels are still labeled as 0, but now each pixle/voxel within an object is labeled as a positive integer. 

In this workflow objects are separated based on connectivity: if a pixel/voxel is touching another pixel/voxel in any direction, they are considered the same object and each pixel/voxel within that object is labeled as the same unique ID number. 

*In the Napari viewer, the image is added as a "labels" layer where each object appears as a different color.*

In [27]:
# create instance segmentation based on connectivity
nuclei_labels = label_uint16(cleaned_img)

# adding image to Napari as a new layer
viewer.add_labels(nuclei_labels, scale=scale, name="7-Nuc: Instance segmentation")

<Labels layer '7-Nuc: Instance segmentation' at 0x1d6e0e91090>

#### **`F. - Use search image to select single nucleus`**

&#x1F453; **FYI:** This code selects a single nucleus label based on which label has the most overlap with the search image.

#### &#x1F6D1; &#x270D; **User Input Required:**

- `Search_Img`: the image/segmentation used to select the nucleus based on its intensities (“Img 5” refers to the mask_I segmentation image and “Img 6” refers to the mask_II segmentation image)

In [28]:
#### USER INPUT REQUIRED ###
Search_Img = "Img 6"

In [29]:
# Collect search image
if Search_Img == "Img 5":
    in_img = mask_I_segmentation
elif Search_Img == "Img 6":
    in_img = mask_II_segmentation

# select single label
keep_nuc = get_max_label((in_img), dilation(nuclei_labels))

# create blank image
nuc_obj = np.zeros_like(nuclei_labels)

# finalize output
nuc_obj[nuclei_labels == keep_nuc] = 1

# adding image to Napari as a new layer
viewer.add_labels(nuc_obj, scale=scale, name="7-Nuc: Select single nucleus")

<Labels layer '7-Nuc: Select single nucleus' at 0x1d6e137fa60>

### **`STEP 8` - Combine nucleus and thresh and fill holes - I**

&#x1F453; **FYI:** This performs a logical **OR** of the nucleus object (can be dilated) and the mask_I_segmentation. After the two segmentations are combined, small objects and holes can be removed. The result of this step is the cellmask_I segmentation.

- To dilate the nucleus in 3D, set Method = `'Ball'`
- To dilate the nucleus in 2D, set Method = `'Disk'`
- To skip dilation, set Method = `'None'`
###### (the string `"None"` is **not** to be confused with the `None` object)

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `Method`: the footprint of the nucleus dilation, "Ball" or "Disk" applies dilation, "None" skips dilation
- `Size`: size of the footprint in the nucleus dilation
- `Min_Hole_Width`: the minimum hole width to be filled in the combined segmentation
- `Max_Hole_Width`: the maximum hole width to be filled in the combined segmentation
- `Small_Obj_Width`: the minimum width of the objects to remain in the combined segmentation

In [30]:
# cm stands for cellmask, helps distinguish variables
cm_method_I = "Disk"
cm_size_I = 0
cm_min_hole_width_I = 0
cm_max_hole_width_I = 25
cm_small_obj_width_I = 1

In [31]:
# create cellmask_I segmentation
cellmask_I_seg = mix_nuc_and_fill(nuc_obj,
                 mask_I_segmentation,
                 Method = cm_method_I,
                 Size = cm_size_I,
                 Min_Hole_Width = cm_min_hole_width_I,
                 Max_Hole_Width = cm_max_hole_width_I,
                 Small_Obj_Width = cm_small_obj_width_I)

# adding image to Napari as a new layer
viewer.add_image(cellmask_I_seg, scale=scale, name="8-Cell: Combine nuc and thresh I")

<Image layer '8-Cell: Combine nuc and thresh I' at 0x1d6e43c3d60>

### **`STEP 9` - Combine nucleus and thresh and fill holes - II**

&#x1F453; **FYI:** This performs a logical **OR** of the nucleus object (can be dilated) and the mask_II_segmentation. After the two segmentations are combined, small objects and holes can be removed. The result of this step is the cellmask_II segmentation.

- To dilate the nucleus in 3D, set Method = `'Ball'`
- To dilate the nucleus in 2D, set Method = `'Disk'`
- To skip dilation, set Method = `'None'`, (the string `"None"` **not** the `None` object)

###### (the string `"None"` is **not** to be confused with the `None` object)

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `Method`: the footprint of the nucleus dilation, "Ball" or "Disk" applies dilation, "None" skips dilation
- `Size`: size of the footprint in the nucleus dilation
- `Min_Hole_Width`: the minimum hole width to be filled in the combined segmentation
- `Max_Hole_Width`: the maximum hole width to be filled in the combined segmentation
- `Small_Obj_Width`: the minimum width of the objects to remain in the combined segmentation

In [32]:
# cm stands for cellmask, helps distinguish variables
cm_method_II = "Disk"
cm_size_II = 0
cm_min_hole_width_II = 0
cm_max_hole_width_II = 25
cm_small_obj_width_II = 1

In [33]:
# create cellmask_II segmentation
cellmask_II_seg = mix_nuc_and_fill(nuc_obj,
                 mask_II_segmentation,
                 Method = cm_method_II,
                 Size = cm_size_II,
                 Max_Hole_Width = cm_min_hole_width_II,
                 Min_Hole_Width = cm_max_hole_width_II,
                 Small_Obj_Width = cm_small_obj_width_II)

# adding image to Napari as a new layer
viewer.add_image(cellmask_II_seg, scale=scale, name="9-Cell: Combine nuc and thresh II")

<Image layer '9-Cell: Combine nuc and thresh II' at 0x1d6da8eb5e0>

-----
## **POST-POST-PROCESSING**

### **`STEP 10` - Watershed for cellmask**

&#x1F453; **FYI:** This step peforms two [watershed](https://scikit-image.org/docs/stable/api/skimage.segmentation.html#skimage.segmentation.watershed) algorithms and then combines the results of both using a logical **OR**. 

For each iteration, the watershed is performed on the mask segmentation (I or II) which is bounded by its respective cellmask segmentation (I or II). To perform the watershed in 3D, set Watershed_Method = `"3D"`, or set Watershed_Method = `"slice_by_slice"` to run it in 2D. Also note that in both iterations, the seed is nucleus segmentation from step 7.

Next, a logical **OR** is used to combine both segmentations to produce an unfinished cellmask. For final touches, the cellmask undergoes a closing (dilation -> erosion) algorithm. Closing can be performed in 3D if Method = `'Ball'`, or 2D if Method = `'Disk'`. Alternatively, the closing algorithm can be skipped altogether if Method = `'None'`. Small objects will be removed after the dilation, but before the erosion.

###### (the string `"None"` is **not** to be confused with the `None` object)

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `Watershed_Method`: whether to perform the watershed on a “3D” level or on a 2D level (“slice_by_slice”)
- `Method`: the footprint of the cellmask closing, "Ball" or "Disk" applies dilation, "None" skips closing
- `Size`: size of the footprint in the cellmask closing
- `Min_Hole_Width`: the minimum hole width to be filled in the cellmask
- `Max_Hole_Width`: the maximum hole width to be filled in the cellmask

In [34]:
# parameters for final cellmask
cm_watershed_method = "3D"
cm_method = "Disk"
cm_size = 15
cm_min_hole_width = 0
cm_max_hole_width = 10000

In [35]:
cellmask_obj = double_watershed(nuc_obj,
                                mask_I_segmentation,
                                mask_II_segmentation,
                                cellmask_I_seg,
                                cellmask_II_seg,
                                Watershed_Method = cm_watershed_method,
                                Min_Hole_Width = cm_min_hole_width,
                                Max_Hole_Width = cm_max_hole_width,
                                Method = cm_method,
                                Size = cm_size)

# adding image to Napari as a new layer
viewer.add_image(cellmask_obj, scale=scale, name="10-Cell: Watershed for Cellmask")

<Image layer '10-Cell: Watershed for Cellmask' at 0x1d6d79fb100>

-----
## **EXPORT**

### **`STEP 11` - Stack masks**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block stacks the nucleus and cell into one multichannel image.

In [36]:
stack = stack_masks(nuc_mask = nuc_obj, cellmask = cellmask_obj)
print(f"Stack mask file structure: {np.shape(stack)}")
print(f"The dimension of '2' represents the 'nucleus' and 'cell' in that order.")

Stack mask file structure: (2, 28, 796, 796)
The dimension of '2' represents the 'nucleus' and 'cell' in that order.


-----
## **SAVING**

### **`Saving` - Save the segmentation output**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block saves the instance segmentation output to the `out_data_path` specified earlier.

In [37]:
out_file_n = export_inferred_organelle(stack, "masks_C", meta_dict, out_data_path)

saved file: 20240118_iN D7 TRC1 ctrl_Z 9_Linear unmixing_0_cmle.ome-masks_C


-----
-----
## **Define functions**
The following code includes an example of how the workflow steps above are combined into functions. The final combined `infer_masks_C` function can be run below to process a single image. It is included in the [batch process notebook](batch_process_segmentations.ipynb) to run the above analysis on multiple cells.

This function can utilized from infer-subc using:
```python
infer_subc.organelles.masks.infer_masks_C()
```

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block defines the `_infer_masks_masks_C()`, function. It is applied below.

In [ ]:
##########################
# _infer_intermediate_masks
# alternative workflow "c"
##########################

def _infer_intermediate_masks(in_img: np.ndarray,
                           pm_ch: Union[int,None],
                           weights_I: list[int],
                           weights_II: list[int],
                           invert_pm_I: bool,
                           invert_pm_II: bool,
                           method_I: str,
                           method_II: str,
                           size_I: int,
                           size_II: int,
                           global_method_I: str,
                           global_method_II: str,
                           cutoff_size_I: int,
                           cutoff_size_II: int,
                           local_adj_I: float,
                           local_adj_II: float,
                           bind_to_pm_I: bool,
                           bind_to_pm_II: bool,
                           thresh_adj_I: float,
                           thresh_adj_II: float):
 
    """
    Procedure to infer intermediate masks from linear unmixed input.

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    pm_ch:
        the index of the channel containing your plasma membrane label
    weights_I:
        a list of int that corresond to the weights for each channel in the first composite; use 0 if a channel should not be included in the composite image
    weights_II:
        a list of int that corresond to the weights for each channel in the second composite; use 0 if a channel should not be included in the composite image
    invert_pm_I:
        True = invert plasma membrane channel in the first composite
        False = do not invert plasma membrane channel in the first composite
    invert_pm_II:
        True = invert plasma membrane channel in the second composite
        False = do not invert plasma membrane channel in the second composite
    method_I:
        which footprint shape to use for the closing algorithm on the first composite. Options include:
        "Ball" (3D closing), "Disk" (2D closing), and "Scharr" (skip closing altogether and apply Scharr edge detection).
    method_II:
        which footprint shape to use for the closing algorithm on the second composite. Options include:
        "Ball" (3D closing), "Disk" (2D closing), and "Scharr" (skip closing altogether and apply Scharr edge detection).
    size_I:
        size of the footprint used in closing algorithm on the first composite, this value is disregarded if method_I == "Scharr"
    size_II:
        size of the footprint used in closing algorithm on the second composite, this value is disregarded if method_II == "Scharr"
    global_method_I:
         which method to use for calculating global threshold applied to composite_mask_I (MO). Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    global_method_II:
         which method to use for calculating global threshold applied to composite_mask_II (MO). Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    cutoff_size_I: 
        Masked Object threshold `size_min`; minimum size of of object to advanced to the local Otsu thresholding
        step in the Masked Object thresholding of composite_mask_I.
    cutoff_size_II: 
        Masked Object threshold `size_min`; minimum size of of object to advanced to the local Otsu thresholding
        step in the Masked Object thresholding of composite_mask_II.
    local_adj_I: 
        Masked Object threshold `local_adjust`, proportion applied to the local Otsu threshold (composite_mask_I MO thresholding)
    local_adj_II: 
        Masked Object threshold `local_adjust`, proportion applied to the local Otsu threshold (composite_mask_II MO thresholding)
    bind_to_pm_I:
        True = restrict the resulting mask_I_segmentation to the thresholded plasma membrane
        False = do not restrict the resulting mask_I_segmentation to the thresholded plasma membrane
    bind_to_pm_II:
        True = restrict the resulting mask_II_segmentation to the thresholded plasma membrane
        False = do not restrict the resulting mask_II_segmentation to the thresholded plasma membrane
    thresh_adj_I:
        the proportion applied to the Otsu threshold of the plasma membrane (disregarded if bind_to_pm_I == False)
    thresh_adj_II:
        the proportion applied to the Otsu threshold of the plasma membrane (disregarded if bind_to_pm_II == False)
    
    

    Returns
    -------------
    mask_I and mask_II segmentation:
        two logical/labels objects that will be used to find the cellmask

    """

    ###################
    # EXTRACT
    ###################

    struct_img_raw_I = membrane_composite(in_img, *weights_I, invert_pm_I, pm_ch)

    struct_img_raw_II = membrane_composite(in_img, *weights_II, invert_pm_II, pm_ch)

    ###################
    # PRE_PROCESSING
    ###################

    composite_mask_I = close_and_filter(struct_img_raw_I, method_I, size_I)

    composite_mask_II = close_and_filter(struct_img_raw_II, method_II, size_II)

    ###################
    # CORE_PROCESSING
    ###################

    mask_I_segmentation = masked_object_thresh_bind_pm(in_img,
                                                       composite_mask_I,
                                                       global_method_I,
                                                       cutoff_size_I,
                                                       local_adj_I,
                                                       bind_to_pm_I,
                                                       pm_ch,
                                                       thresh_adj_I)
    
    mask_II_segmentation = masked_object_thresh_bind_pm(in_img,
                                                       composite_mask_II,
                                                       global_method_II,
                                                       cutoff_size_II,
                                                       local_adj_II,
                                                       bind_to_pm_II,
                                                       pm_ch,
                                                       thresh_adj_II)
    
    return label_bool_as_uint16(mask_I_segmentation),label_bool_as_uint16(mask_II_segmentation)


#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block applies the function above to your test image. The settings specified above are applied here.

In [39]:
# run nuclei segmentation from label
_mask_I_segmentation,_mask_II_segmentation = _infer_intermediate_masks(img_data,
                           PM_CH,
                           [w0_I, w1_I, w2_I, w3_I, w4_I, w5_I, w6_I, w7_I, w8_I, w9_I],
                           [w0_II, w1_II, w2_II, w3_II, w4_II, w5_II, w6_II, w7_II, w8_II, w9_II],
                           invert_pm_I,
                           invert_pm_II,
                           method_I,
                           method_II,
                           size_I,
                           size_II,
                           global_method_I,
                           global_method_II,
                           cutoff_size_I,
                           cutoff_size_II,
                           local_adj_I,
                           local_adj_II,
                           bind_to_pm_I,
                           bind_to_pm_II,
                           thresh_adj_I,
                           thresh_adj_II)

# confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(mask_I_segmentation == _mask_I_segmentation) & np.all(mask_II_segmentation == _mask_II_segmentation)}")

The segmentation output here matches the output created above: True


#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block defines the `_infer_nucleus_masks_C()`, function. It is applied below.

In [ ]:
##########################
# _infer_nucleus_masks_C
# alternative workflow "c"
##########################

def _infer_nucleus_masks_C(in_img: np.ndarray,
                           nuc_ch: Union[int,None],
                           mask_I_segmentation: np.ndarray,
                           mask_II_segmentation: np.ndarray,
                           nuc_med_filter_size: int,
                           nuc_gaussian_smoothing_sigma: float,
                           nuc_threshold_factor: float,
                           nuc_thresh_min: float,
                           nuc_thresh_max: float,
                           nuc_hole_min_width: int,
                           nuc_hole_max_width: int,
                           nuc_small_object_width: int,
                           nuc_fill_filter_method: str,
                           nuc_search_img: str):
    
    """
    Procedure to infer intermediate masks from linear unmixed input.

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    nuc_ch:
        the index of the channel containing your nuclei label
    mask_I_segmentation:
        the first logical/labels object that will be used to find the cellmask
    mask_II_segmentation:
        the second logical/labels object that will be used to find the cellmask
    nuc_med_filter_size: 
        width of median filter for nuclei signal
    nuc_gaussian_smoothing_sigma: 
        sigma for gaussian smoothing of nuclei signal
    nuc_threshold_factor:
        adjustment to make to the intial local threshold of the nucleus singal;
        intital threshold is derived from Li's minimum cross entropy method
    nuc_thresh_min:
        minimum bound of the local threshold of the nucleus
    nuc_thresh_max:
        maximum bound of the local threshold of the nucleus
    nuc_hole_min_width: 
        the minimum hole width to be filled in the nucleus post-thresholding
    nuc_hole_max_width:
        the maximum hole width to be filled in the nucleus post-thresholding
    nuc_small_object_width:
        minimum object size cutoff for nucleus post-thresholding
    nuc_fill_filter_method:
        determines if fill and filter should be run 'sice-by-slice' or in '3D'
    nuc_search_img:
        the segmentation used to select the nucleus based on greatest overlap. Options include:
        "Img 5" (mask_I_segmentation) and "Img 6" (mask_II_segmentation)
    
    Returns
    -------------
    nucleus_object:
        mask defined extent of NU

    """
    ###################
    # POST_PROCESSING
    ###################

    nuc_obj = find_nuc(in_img, 
             mask_I_segmentation,
             mask_II_segmentation,
             nuc_ch,
             nuc_med_filter_size,
             nuc_gaussian_smoothing_sigma,
             nuc_threshold_factor,
             nuc_thresh_min,
             nuc_thresh_max,
             nuc_hole_min_width,
             nuc_hole_max_width,
             nuc_small_object_width,
             nuc_fill_filter_method,
             nuc_search_img)
    
    return label_bool_as_uint16(nuc_obj)

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block applies the function above to your test image. The settings specified above are applied here.

In [41]:
# run nuclei segmentation from label
_NU_object = _infer_nucleus_masks_C(img_data,
                           NUC_CH,
                           _mask_I_segmentation,
                           _mask_II_segmentation,
                           nuc_med_filter_size,
                           nuc_gaussian_smoothing_sigma,
                           nuc_threshold_factor,
                           nuc_thresh_min,
                           nuc_thresh_max,
                           nuc_hole_min_width,
                           nuc_hole_max_width,
                           nuc_small_object_width,
                           nuc_fill_filter_method,
                           Search_Img)

# confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(nuc_obj == _NU_object)}")

The segmentation output here matches the output created above: True


#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block defines the `_infer_cellmask_masks_C()`, function. It is applied below.

In [42]:
##########################
# _infer_cellmask_masks_C
# alternative workflow "c"
##########################
def _infer_cellmask_masks_C(nuc_obj: np.ndarray,
                           mask_I_segmentation: np.ndarray,
                           mask_II_segmentation: np.ndarray,
                           cm_method_I: str,
                           cm_method_II: str,
                           cm_size_I: int,
                           cm_size_II: int,
                           cm_min_hole_width_I: int,
                           cm_min_hole_width_II: int,
                           cm_max_hole_width_I: int,
                           cm_max_hole_width_II: int,
                           cm_small_obj_width_I: int,
                           cm_small_obj_width_II: int,
                           cell_watershed_method: str,
                           cell_min_hole_width: int,
                           cell_max_hole_width: int,
                           cell_method: str,
                           cell_size: int):
    
    """
    Procedure to infer cellmask from linear unmixed input.

    Parameters
    ------------
    nuc_obj:
        a 3d image containing the inferred nucleus object
    mask_I_segmentation:
        the first logical/labels object that will be used to find the cellmask
    mask_II_segmentation:
        the second logical/labels object that will be used to find the cellmask
    cm_method_I:
        which footprint shape to use for the nucleus dilation before combination with the mask_I_segmentation. Options include:
        "Ball" (3D dilation), "Disk" (2D dilation), and "None" (skip dilation altogether).
    cm_method_II:
        which footprint shape to use for the nucleus dilation before combination with the mask_II_segmentation. Options include:
        "Ball" (3D dilation), "Disk" (2D dilation), and "None" (skip dilation altogether).
    cm_size_I:
        size of the footprint used in dilation of the nucleus object, this value is disregarded if cm_method_I == "None"
    cm_size_II:
        size of the footprint used in dilation of the nucleus object, this value is disregarded if cm_method_II == "None"
    cm_min_hole_width_I: 
        the minimum hole width to be filled in the cellmask_I_segmentation
    cm_min_hole_width_II: 
        the minimum hole width to be filled in the cellmask_II_segmentation
    cm_max_hole_width_I: 
        the maximum hole width to be filled in the cellmask_I_segmentation
    cm_max_hole_width_II: 
        the maximum hole width to be filled in the cellmask_II_segmentation
    cm_small_obj_width_I:
        minimum object size cutoff for the cellmask_I_segmentation
    cm_small_obj_width_II:
        minimum object size cutoff for the cellmask_II_segmentation
    cell_watershed_method:
        determines if the watershed should be run 'sice-by-slice' or in '3D'
    cell_min_hole_width: 
        the minimum hole width to be filled in the cellmask object
    cell_max_hole_width: 
        the maximum hole width to be filled in the cellmask object
    cell_method:
        which footprint shape to use for the cellmask closing (dilation -> erosion). Options include:
        "Ball" (3D closing), "Disk" (2D closing), and "None" (skip closing altogether).
    cell_size:
        size of the footprint used in closing of the cellmask object, this value is disregarded if cm_method_I == "None"
        
    
    Returns
    -------------
    cellmask:
        a logical/labels object defining boundaries of cellmask

    """
    ###################
    # POST_PROCESSING
    ###################

    cellmask_I_seg = mix_nuc_and_fill(nuc_obj,
                 mask_I_segmentation,
                 cm_method_I,
                 cm_size_I,
                 cm_min_hole_width_I,
                 cm_max_hole_width_I,
                 cm_small_obj_width_I)
    
    cellmask_II_seg = mix_nuc_and_fill(nuc_obj,
                 mask_II_segmentation,
                 cm_method_II,
                 cm_size_II,
                 cm_min_hole_width_II,
                 cm_max_hole_width_II,
                 cm_small_obj_width_II)
    
    #########################
    # POST- POST_PROCESSING
    #########################

    cellmask_obj = double_watershed(nuc_obj,
                                mask_I_segmentation,
                                mask_II_segmentation,
                                cellmask_I_seg,
                                cellmask_II_seg,
                                cell_watershed_method,
                                cell_min_hole_width,
                                cell_max_hole_width,
                                cell_method,
                                cell_size)
    
    return label_bool_as_uint16(cellmask_obj)

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block applies the function above to your test image. The settings specified above are applied here.

In [43]:
# run nuclei segmentation from label
_CM_object = _infer_cellmask_masks_C(_NU_object,
                                    _mask_I_segmentation,
                                    _mask_II_segmentation,
                                    cm_method_I,
                                    cm_method_II,
                                    cm_size_I,
                                    cm_size_II,
                                    cm_min_hole_width_I,
                                    cm_min_hole_width_II,
                                    cm_max_hole_width_I,
                                    cm_max_hole_width_II,
                                    cm_small_obj_width_I,
                                    cm_small_obj_width_II,
                                    cm_watershed_method,
                                    cm_min_hole_width,
                                    cm_max_hole_width,
                                    cm_method,
                                    cm_size)

# confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(cellmask_obj == _CM_object)}")

The segmentation output here matches the output created above: True


#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block defines the `infer_masks_C()`, function, the complete function that combines the three above and that will be used for batch processing. It is applied below.

In [ ]:
def _infer_masks_C(in_img: np.ndarray,
                   pm_ch: Union[int,None],
                   nuc_ch: Union[int,None],
                   weights_I: list[int],
                   weights_II: list[int],
                   invert_pm_I: bool,
                   invert_pm_II: bool,
                   method_I: str,
                   method_II: str,
                   size_I: int,
                   size_II: int,
                   global_method_I: str,
                   global_method_II: str,
                   cutoff_size_I: int,
                   cutoff_size_II: int,
                   local_adj_I: float,
                   local_adj_II: float,
                   bind_to_pm_I: bool,
                   bind_to_pm_II: bool,
                   thresh_adj_I: float,
                   thresh_adj_II: float,
                   nuc_med_filter_size: int,
                   nuc_gaussian_smoothing_sigma: float,
                   nuc_threshold_factor: float,
                   nuc_thresh_min: float,
                   nuc_thresh_max: float,
                   nuc_hole_min_width: int,
                   nuc_hole_max_width: int,
                   nuc_small_object_width: int,
                   nuc_fill_filter_method: str,
                   nuc_search_img: str,
                   cm_method_I: str,
                   cm_method_II: str,
                   cm_size_I: int,
                   cm_size_II: int,
                   cm_min_hole_width_I: int,
                   cm_min_hole_width_II: int,
                   cm_max_hole_width_I: int,
                   cm_max_hole_width_II: int,
                   cm_small_obj_width_I: int,
                   cm_small_obj_width_II: int,
                   cell_watershed_method: str,
                   cell_min_hole_width: int,
                   cell_max_hole_width: int,
                   cell_method: str,
                   cell_size: int) -> np.ndarray:
    """
    Procedure to infer intermediate masks from linear unmixed input.

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    pm_ch:
        the index of the channel containing your plasma membrane label
    nuc_ch:
        the index of the channel containing your nuclei label
    weights_I:
        a list of int that corresond to the weights for each channel in the first composite; use 0 if a channel should not be included in the composite image
    weights_II:
        a list of int that corresond to the weights for each channel in the second composite; use 0 if a channel should not be included in the composite image
    invert_pm_I:
        True = invert plasma membrane channel in the first composite
        False = do not invert plasma membrane channel in the first composite
    invert_pm_II:
        True = invert plasma membrane channel in the second composite
        False = do not invert plasma membrane channel in the second composite
    method_I:
        which footprint shape to use for the closing algorithm on the first composite. Options include:
        "Ball" (3D closing), "Disk" (2D closing), and "Scharr" (skip closing altogether and apply Scharr edge detection).
    method_II:
        which footprint shape to use for the closing algorithm on the second composite. Options include:
        "Ball" (3D closing), "Disk" (2D closing), and "Scharr" (skip closing altogether and apply Scharr edge detection).
    size_I:
        size of the footprint used in closing algorithm on the first composite, this value is disregarded if method_I == "Scharr"
    size_II:
        size of the footprint used in closing algorithm on the second composite, this value is disregarded if method_II == "Scharr"
    global_method_I:
         which method to use for calculating global threshold applied to composite_mask_I (MO). Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    global_method_II:
         which method to use for calculating global threshold applied to composite_mask_II (MO). Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    cutoff_size_I: 
        Masked Object threshold `size_min`; minimum size of of object to advanced to the local Otsu thresholding
        step in the Masked Object thresholding of composite_mask_I.
    cutoff_size_II: 
        Masked Object threshold `size_min`; minimum size of of object to advanced to the local Otsu thresholding
        step in the Masked Object thresholding of composite_mask_II.
    local_adj_I: 
        Masked Object threshold `local_adjust`, proportion applied to the local Otsu threshold (composite_mask_I MO thresholding)
    local_adj_II: 
        Masked Object threshold `local_adjust`, proportion applied to the local Otsu threshold (composite_mask_II MO thresholding)
    bind_to_pm_I:
        True = restrict the resulting mask_I_segmentation to the thresholded plasma membrane
        False = do not restrict the resulting mask_I_segmentation to the thresholded plasma membrane
    bind_to_pm_II:
        True = restrict the resulting mask_II_segmentation to the thresholded plasma membrane
        False = do not restrict the resulting mask_II_segmentation to the thresholded plasma membrane
    thresh_adj_I:
        the proportion applied to the Otsu threshold of the plasma membrane (disregarded if bind_to_pm_I == False)
    thresh_adj_II:
        the proportion applied to the Otsu threshold of the plasma membrane (disregarded if bind_to_pm_II == False)
    nuc_med_filter_size: 
        width of median filter for nuclei signal
    nuc_gaussian_smoothing_sigma: 
        sigma for gaussian smoothing of nuclei signal
    nuc_threshold_factor:
        adjustment to make to the intial local threshold of the nucleus singal;
        intital threshold is derived from Li's minimum cross entropy method
    nuc_thresh_min:
        minimum bound of the local threshold of the nucleus
    nuc_thresh_max:
        maximum bound of the local threshold of the nucleus
    nuc_hole_min_width: 
        the minimum hole width to be filled in the nucleus post-thresholding
    nuc_hole_max_width:
        the maximum hole width to be filled in the nucleus post-thresholding
    nuc_small_object_width:
        minimum object size cutoff for nucleus post-thresholding
    nuc_fill_filter_method:
        determines if fill and filter should be run 'sice-by-slice' or in '3D'
    nuc_search_img:
        the segmentation used to select the nucleus based on greatest overlap. Options include:
        "Img 5" (mask_I_segmentation) and "Img 6" (mask_II_segmentation)
    cm_method_I:
        which footprint shape to use for the nucleus dilation before combination with the mask_I_segmentation. Options include:
        "Ball" (3D dilation), "Disk" (2D dilation), and "None" (skip dilation altogether).
    cm_method_II:
        which footprint shape to use for the nucleus dilation before combination with the mask_II_segmentation. Options include:
        "Ball" (3D dilation), "Disk" (2D dilation), and "None" (skip dilation altogether).
    cm_size_I:
        size of the footprint used in dilation of the nucleus object, this value is disregarded if cm_method_I == "None"
    cm_size_II:
        size of the footprint used in dilation of the nucleus object, this value is disregarded if cm_method_II == "None"
    cm_min_hole_width_I: 
        the minimum hole width to be filled in the cellmask_I_segmentation
    cm_min_hole_width_II: 
        the minimum hole width to be filled in the cellmask_II_segmentation
    cm_max_hole_width_I: 
        the maximum hole width to be filled in the cellmask_I_segmentation
    cm_max_hole_width_II: 
        the maximum hole width to be filled in the cellmask_II_segmentation
    cm_small_obj_width_I:
        minimum object size cutoff for the cellmask_I_segmentation
    cm_small_obj_width_II:
        minimum object size cutoff for the cellmask_II_segmentation
    cell_watershed_method:
        determines if the watershed should be run 'sice-by-slice' or in '3D'
    cell_min_hole_width: 
        the minimum hole width to be filled in the cellmask object
    cell_max_hole_width: 
        the maximum hole width to be filled in the cellmask object
    cell_method:
        which footprint shape to use for the cellmask closing (dilation -> erosion). Options include:
        "Ball" (3D closing), "Disk" (2D closing), and "None" (skip closing altogether).
    cell_size:
        size of the footprint used in closing of the cellmask object, this value is disregarded if cm_method_I == "None"
    
    Returns
    -------------
    mask_stack:
        a two channel np.ndarray constisting of the nucleus and cell (one object per channel)

    """

    ##########################
    # get intermediate masks #
    ##########################

    mask_I_segmentation, mask_II_segmentation = _infer_intermediate_masks(in_img,
                           pm_ch,
                           weights_I,
                           weights_II,
                           invert_pm_I,
                           invert_pm_II,
                           method_I,
                           method_II,
                           size_I,
                           size_II,
                           global_method_I,
                           global_method_II,
                           cutoff_size_I,
                           cutoff_size_II,
                           local_adj_I,
                           local_adj_II,
                           bind_to_pm_I,
                           bind_to_pm_II,
                           thresh_adj_I,
                           thresh_adj_II)
    
    #################
    # infer nucleus #
    #################
    
    nuc_obj = _infer_nucleus_masks_C(in_img,
                           nuc_ch,
                           mask_I_segmentation,
                           mask_II_segmentation,
                           nuc_med_filter_size,
                           nuc_gaussian_smoothing_sigma,
                           nuc_threshold_factor,
                           nuc_thresh_min,
                           nuc_thresh_max,
                           nuc_hole_min_width,
                           nuc_hole_max_width,
                           nuc_small_object_width,
                           nuc_fill_filter_method,
                           nuc_search_img)
    
    ##################
    # infer cellmask #
    ##################
    
    cellmask_obj = _infer_cellmask_masks_C(nuc_obj,
                                    mask_I_segmentation,
                                    mask_II_segmentation,
                                    cm_method_I,
                                    cm_method_II,
                                    cm_size_I,
                                    cm_size_II,
                                    cm_min_hole_width_I,
                                    cm_min_hole_width_II,
                                    cm_max_hole_width_I,
                                    cm_max_hole_width_II,
                                    cm_small_obj_width_I,
                                    cm_small_obj_width_II,
                                    cell_watershed_method,
                                    cell_min_hole_width,
                                    cell_max_hole_width,
                                    cell_method,
                                    cell_size)
    
    ###################
    ### stack masks ###
    ###################
    mask_stack = stack_masks(nuc_mask=nuc_obj, 
                            cellmask=cellmask_obj)
    
    return mask_stack

In [45]:
_stacked_masks = _infer_masks_C(img_data,
                                PM_CH,
                                NUC_CH,
                                [w0_I, w1_I, w2_I, w3_I, w4_I, w5_I, w6_I, w7_I, w8_I, w9_I],
                                [w0_II, w1_II, w2_II, w3_II, w4_II, w5_II, w6_II, w7_II, w8_II, w9_II],
                                invert_pm_I,
                                invert_pm_II,
                                method_I,
                                method_II,
                                size_I,
                                size_II,
                                global_method_I,
                                global_method_II,
                                cutoff_size_I,
                                cutoff_size_II,
                                local_adj_I,
                                local_adj_II,
                                bind_to_pm_I,
                                bind_to_pm_II,
                                thresh_adj_I,
                                thresh_adj_II,
                                nuc_med_filter_size,
                                nuc_gaussian_smoothing_sigma,
                                nuc_threshold_factor,
                                nuc_thresh_min,
                                nuc_thresh_max,
                                nuc_hole_min_width,
                                nuc_hole_max_width,
                                nuc_small_object_width,
                                nuc_fill_filter_method,
                                Search_Img,
                                cm_method_I,
                                cm_method_II,
                                cm_size_I,
                                cm_size_II,
                                cm_min_hole_width_I,
                                cm_min_hole_width_II,
                                cm_max_hole_width_I,
                                cm_max_hole_width_II,
                                cm_small_obj_width_I,
                                cm_small_obj_width_II,
                                cm_watershed_method,
                                cm_min_hole_width,
                                cm_max_hole_width,
                                cm_method,
                                cm_size)

#confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(stack == _stacked_masks)}")

The segmentation output here matches the output created above: True


-------------
### ✅ **INFER MASKS COMPLETE!**

Now that a single cell has been identified for analysis, the organelle segmentions need to be completed. Continue on to complete the organelle notebooks (1.2 through 1.7) associated to each of the organelles you would like to include in your analysis:
- Infer [`lysosomes`](1.2_infer_lysosome.ipynb)
- Infer [`mitochondria`](1.3_infer_mitochondria.ipynb)
- Infer [`golgi`](1.4_infer_golgi.ipynb)
- Infer [`peroxisomes`](1.5_infer_peroxisome.ipynb)
- Infer [`endoplasmic reticulum (ER)`](1.6_infer_ER.ipynb)
- Infer [`lipid droplets`](1.7_infer_lipid_droplet.ipynb)